In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## Preparando os dados

In [ ]:
# Carregando os dados
df = pd.read_csv('../input/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.head()

In [ ]:
# Tamanho do dataframe e tipos dos dados
df.info()

In [ ]:
# Limpando os valores em branco, trocando espaço em branco por -1, e convertendo a coluna
df['TotalCharges'] = df['TotalCharges'].str.strip().replace('', '-1').astype(float)

In [ ]:
# Convertendo as colunas object para colunas categóricas
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category').cat.codes

## Rodando o Modelo

In [ ]:
# Separando o dataframe
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.20, random_state=42)

In [ ]:
# Separando os dados para validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

In [ ]:
# Verificando os tamanhos
train.shape, valid.shape, test.shape

In [ ]:
# Lista das colunas a serem usadas para treino
feats = [c for c in df.columns if c not in ['customerID', 'Churn']]

In [ ]:
# Importando o RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Instanciando um objeto RandomForest
rf = RandomForestClassifier(n_estimators=200, min_samples_split=5, max_depth=4,
                            random_state=42)

In [ ]:
# Treinando o modelo
rf.fit(train[feats], train['Churn'])

In [ ]:
# Fazer previsões usando o modelo treinado
preds = rf.predict(valid[feats])

## Avaliando o desempenho do modelo

In [ ]:
# Importando a métrica
from sklearn.metrics import accuracy_score

In [ ]:
# Avaliando com relação aos dados de validação
accuracy_score(valid['Churn'], preds)

In [ ]:
# Avaliando o modelo com relação aos dados de teste
accuracy_score(test['Churn'], rf.predict(test[feats]))

In [ ]:
# Avaliando a importância das variaveis
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

## Confusion Matrix

In [ ]:
# Importando uma biblioteca específica para plotar gráficos relacionada
# com o scikitlearn
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de validação
skplt.metrics.plot_confusion_matrix(valid['Churn'], preds)

In [ ]:
# Matriz de Confusão - Dados de teste
skplt.metrics.plot_confusion_matrix(test['Churn'], rf.predict(test[feats]))

## Precision / Recall

In [ ]:
# Função para plotar a curva de Precision/Recall
import matplotlib.pyplot as plt
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="upper left")
    plt.ylim([0, 1])

In [ ]:
# Gerando os dados para a curva - Dados de validação
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(valid['Churn'],
                                                         rf.predict_proba(valid[feats]).T[1])

In [ ]:
rf.predict_proba(valid[feats])

In [ ]:
# Plotando a curva - Dados de validação
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)

In [ ]:
# Gerando os dados para a curva - Dados de teste
precisions, recalls, thresholds = precision_recall_curve(test['Churn'],
                                                         rf.predict_proba(test[feats]).T[1])

In [ ]:
# Plotando a curva - Dados de teste
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)

In [ ]:
import matplotlib.pyplot as plt

probas = rf.predict_proba(test[feats])

skplt.metrics.plot_precision_recall(test['Churn'], probas)
plt.show()